In [62]:
import brightway2 as bw
from bw2data.parameters import*
import numpy as np
from SALib.sample import saltelli
from SALib.analyze import sobol

# We create an new DB and a power generation activity
### Very simple, the power generation activity does not have an input but only a CO2 emission and an electricity output
### the CO2 depends here on the power plant efficiency, the fuel LHV, CO2 intensity and its biomass content

In [63]:
bw.projects.set_current("power_plant_example")
bw.bw2setup()

database = bw.Database("db")
power_plant_data = {
    ("db", "electricity power plant"): {
        "name": "electricity production",
        'unit': 'kWh',
        'location': 'CH',
        "exchanges": [{
            "amount": 0.5,
            "input": ('biosphere3', 'f9749677-9c9f-4678-ab55-c607dfdc2cb9'), #Carbon dioxide, fossil
            "type": "biosphere",
            "uncertainty type":0,
            "unit": "kg",
            "formula":"(3.6/conversion_efficiency)*(fuel_CO2_intensity/fuel_LHV)*(1-fuel_biomass_share)"} #this is the formula containing parameters
        ,
        {
            "amount": 1,
            "input": ('db', 'electricity power plant'), #1 kWh of electricity
            "type": "production",
            "uncertainty type":0,
            "unit": "kWh"}
        ]}}
database.write(power_plant_data)

Writing activities to SQLite3 database:
0%  100%
[#] | ETA: 00:00:00
Total time elapsed: 00:00:00


Biosphere database already present!!! No setup is needed
Title: Writing activities to SQLite3 database:
  Started: 11/26/2018 10:49:55
  Finished: 11/26/2018 10:49:55
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 0.63


# we select a method for characterization
# we declare our parameters to BW and where they can be found
# we gather the parameters in a parameter group and ask BW to refresh the exchange values

In [64]:
method = ('CML 2001', 'climate change', 'GWP 100a')
activity_data=[
        {"name":"fuel_LHV", "amount":15, "database":"db", "code":"electricity power plant"},
        {"name":"fuel_CO2_intensity", "amount":1.5, "database":"db", "code":"electricity power plant"},
        {"name":"fuel_biomass_share", "amount":.25, "database":"db", "code":"electricity power plant"},
        {"name":"conversion_efficiency", "amount":.6, "database":"db", "code":"electricity power plant"}]
parameters.new_activity_parameters(activity_data, "act_param", overwrite=True)
parameters.add_exchanges_to_group("act_param", bw.Database("db").get("electricity power plant"))
ActivityParameter.recalculate_exchanges("act_param")

# We create a BW calculation function that updates the parameters values in the exchanges and returns a CO2 value

In [65]:
def bw_calc(param_list):
    ActivityParameter.update(amount=param_list[0]).where(ActivityParameter.name == "fuel_LHV").execute()
    ActivityParameter.update(amount=param_list[1]).where(ActivityParameter.name == "fuel_CO2_intensity").execute()
    ActivityParameter.update(amount=param_list[2]).where(ActivityParameter.name == "fuel_biomass_share").execute()
    ActivityParameter.update(amount=param_list[3]).where(ActivityParameter.name == "conversion_efficiency").execute()
    ActivityParameter.recalculate_exchanges("act_param")
    my_lca=bw.LCA({bw.Database("db").get('electricity power plant'):1}, method)
    my_lca.lci()
    my_lca.lcia()
    return my_lca.score  

# We declare our problem space: the parameters as well as the boundary values we think they might take (e.g. 95% CI).

In [66]:
problem = {
    'num_vars': 4,
    'names': ['fuel_LHV', 'fuel_CO2_intensity', 'fuel_biomass_share', 'conversion_efficiency'],
    'bounds': [[10, 15],
               [1.5, 2],
               [.2, .4],
               [.5, .7]]
}

# we generate a sobol sequence from the problem space given.
## here, the argument given is 100,meaning it will generate 100*(2*4+2) = 1000 samples, each of them contains four random values for our 4 parameters.

In [67]:
param_values = saltelli.sample(problem, 100, calc_second_order=True)
Y = np.zeros([param_values.shape[0]])

# we feed the Sobol sequence to the BW function and retrieve the results in an array (Y). This step can take a while.

In [68]:
for i, X in enumerate(param_values):
    Y[i] = bw_calc(X)

# we calculate the Sobol indices

In [69]:
Si = sobol.analyze(problem, Y, calc_second_order=True)
#display first, second and total order indices
#make sure to have samples large enough to give meaningful second order relations
print(Si)

{'S1': array([ 0.31644445,  0.17680569,  0.17840504,  0.24053761]), 'S1_conf': array([ 0.15427923,  0.12385726,  0.12503546,  0.14691116]), 'ST': array([ 0.32294062,  0.17776315,  0.18750861,  0.24183004]), 'ST_conf': array([ 0.10693683,  0.05529663,  0.04797578,  0.06458563]), 'S2': array([[        nan,  0.01073416,  0.00250264, -0.01441787],
       [        nan,         nan,  0.01648918, -0.03710391],
       [        nan,         nan,         nan, -0.03093666],
       [        nan,         nan,         nan,         nan]]), 'S2_conf': array([[        nan,  0.22470228,  0.21775779,  0.21626284],
       [        nan,         nan,  0.19717882,  0.21309387],
       [        nan,         nan,         nan,  0.18631655],
       [        nan,         nan,         nan,         nan]])}


In [70]:
Si["ST"]

array([ 0.32294062,  0.17776315,  0.18750861,  0.24183004])

In [71]:
Si["S1"]

array([ 0.31644445,  0.17680569,  0.17840504,  0.24053761])